In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from vitrocal.datasets import catalog
from vitrocal.preprocessors import StandardPreprocessor
from vitrocal.detectors import DerivativeDetector, StandardExtractor
from vitrocal.analyzers import StandardAnalyzer

In [ ]:
# to load actual data

datacatalog = catalog.DataCatalog()
df = datacatalog.load('figi_output')
# df = pd.read_csv("file.csv")

df

In [ ]:
preprocessor = StandardPreprocessor(
    frames_per_second=1/2.5, 
    bleach_period=60,
    filter_frequency=.15, 
    baseline_threshold=10, 
    window_size=60)

data = preprocessor.drop_frames(df)
data

In [ ]:
preprocessor.filter(data)

In [ ]:
baseline = preprocessor.baseline(data)
baseline

In [ ]:
d_f = preprocessor.compute_fluoresence_change(data, baseline)

In [ ]:
# alternatively:
d_f = preprocessor.preprocess(df)
d_f

In [ ]:
detector = DerivativeDetector(
    threshold=20
)

detected = detector.detect(d_f)
detected.head(15)

In [ ]:
extractor = StandardExtractor(
    window = (3, 30), # this is in seconds
    frames_per_second=1/2.5, 
    threshold=20)

In [ ]:
# calls DerivativeDetector.detect()
events = extractor.detect_and_extract(d_f)
events

In [ ]:
analyzer = StandardAnalyzer(
    upper_decay_bound=.8,
    lower_decay_bound=.2
)

results, avg_results = analyzer.analyze(events)
results

In [ ]:
avg_results

In [ ]:
# examine a few events
arbitrary_events = range(1, 10+1)

fig, axs = plt.subplots(int(len(arbitrary_events)/2), 
                        ncols=2,
                        figsize=(10, 8))

for roi, ax in zip(arbitrary_events, axs.flat):

    to_plot = random.choice(events[roi])
    ax.plot(to_plot)
    ax.set_title('ROI:' + str(roi))

fig.tight_layout()
plt.show()


In [ ]:
# want the index-wise average

combined = pd.Series()
for roi, values in events.items():
    for sequence in values:
        tmp = pd.Series(sequence)
        combined = pd.concat([combined, tmp], axis=1).agg("mean", axis=1)

plt.plot(combined.sort_index())
plt.title("'Average' Detected Event")
